# Deep Learning  第6回

## 今回の目標
前回実装したニューラルネットワークの学習機能を誤差逆伝播法を用いて高速化する

## 誤差逆伝播法
前回実装したニューラルネットワークの学習では勾配計算に特に時間がかかっていた。  
今回は誤差逆伝播法を用いることで勾配計算を効率よく行う。

1. サンプルデータを取る
2.  損失を算出する
3.  **勾配を算出する** <- ここの高速化
4.  重みを修正
5.  繰り返す

### 誤差逆伝播を求める手順
1. 順伝播を求める
2. 逆伝播を求める

### 順伝播
局所的な計算を順番に行う

### 計算グラフ
計算過程をグラフで表したもの  
これを使うと順伝搬、逆伝搬が視覚的に理解しやすくなる

例）
１個100円のグレープフルーツを２個購入したときの合計金額を求めよ。ただし、消費税10%が適用されるものとする。

![](images/backpropagation/computational-graph.svg)

![](images/backpropagation/computational-graph2-2.svg)

![](images/backpropagation/backpropagation.svg)

### 乗算レイヤ

$t=x\times y\\ \frac { \partial t }{ \partial x } =y\\ \frac { \partial t }{ \partial y } =x$

In [6]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [9]:
grapefruit = 100
grapefruit_num = 2
tax = 1.1

mul_grapefruit_layer = MulLayer()
mul_tax_layer = MulLayer()

# 順伝播
grapefruit_price = mul_grapefruit_layer.forward(grapefruit, grapefruit_num)
price = mul_tax_layer.forward(grapefruit_price, tax)

print(price)

220.00000000000003


In [21]:
dprice = 1

# 逆伝播
dgrapefruit_price, dtax = mul_tax_layer.backward(dprice)
dgrapefruit, dgrapefruit_num = mul_grapefruit_layer.backward(dgrapefruit_price)

print(dgrapefruit, dgrapefruit_num, dtax)

2.2 110.00000000000001 200


### 加算レイヤ

$$t=x+y\\ \frac { \partial t }{ \partial x } =1\\ \frac { \partial t }{ \partial y } =1$$

In [14]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

### シグモイドレイヤ

#### シグモイド関数

$$h(x)=\frac { 1 }{ 1+exp(-x) } $$

![](images/backpropagation/computational-graph-sigmoid.svg)

![](images/backpropagation/computational-graph-sigmoid-function.svg)

$$\frac { \partial L }{ \partial y } { y }^{ 2 }exp(-x)\\ =\frac { \partial L }{ \partial y } \frac { 1 }{ { (1+exp(-x)) }^{ 2 } } exp(-x)\\ =\frac { \partial L }{ \partial y } \frac { 1 }{ { 1+exp(-x) } } \frac { exp(-x) }{ 1+exp(-x) } \\ =\frac { \partial L }{ \partial y } y(1-y)$$

In [15]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
      dx = dout * (1.0 + self.out) * self.out

      return dx

### ReLUレイヤ

#### ReLU関数

$$h(x)=\begin{cases} 0\quad (x\le \theta ) \\ x\quad (x>\theta ) \end{cases}$$

![](images/backpropagation/computational-graph-relu.svg)

In [16]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx